# Necessary Common Functions

Those functions should be ran before each part.

In [1]:
import os
import random
import numpy as np
import pandas as pd
import cudf, itertools
import scipy.sparse as ssp
from functools import lru_cache
from tqdm import tqdm, trange
from collections import Counter, defaultdict
import numba 
from numba import jit
import datasets
from datasets import Dataset as TFDataset
import multiprocessing

In [2]:
def get_sessions(df: pd.DataFrame, test=False, list_item=False) -> list:
    
    all_item = []
    if 'next_item' in df and not test:
        if list_item:
            for i in trange(len(df)):
                all_item.append(np.concatenate([np.array(df.loc[i, 'prev_items']), np.array(df.loc[i, 'next_item'])], axis=0))
        else:
            for i in trange(len(df)):
                all_item.append(eval((df.loc[i, 'prev_items'][:-1]+f" '{df.loc[i, 'next_item']}']").replace(" ", ",")))
    else:
        if list_item:
            all_item = df['prev_items']
        else:
            for i in trange(len(df)):
                all_item.append(eval((df.loc[i, 'prev_items']).replace(" ", ",")))
    return all_item

In [3]:
def get_co_occurence_dict(sessions: list, bidirection: bool=True, weighted: bool=False, max_dis=None) -> dict:
    res = {}
    for sess in tqdm(sessions):
        for i, id in enumerate(sess):
            if id not in res:
                res[id] = Counter()
            
            if max_dis == None:
                e = len(sess)
            else:
                e = min(i + max_dis + 1, len(sess))

            for j in range(i+1, e):
                if not weighted:
                    res[id][sess[j]] += 1
                else:
                    res[id][sess[j]] += 1 / (j-i)
                if bidirection:
                    if sess[j] not in res:
                        res[sess[j]] = Counter()
                    if not weighted:
                        res[sess[j]][id] += 1
                    else:
                        res[sess[j]][id] += 1 / (j-i)
    return res

In [4]:
def sort_co_occurence_dict(co_occurence_dict: dict) -> dict:
    res = {}
    for k,v in co_occurence_dict.items():
        res[k] = dict(sorted(v.items(), key=lambda item: -item[1]))
    return res

In [5]:
def cast_dtype(df : pd.DataFrame, columns=None):
    if columns is None:
        columns = df.columns
    for k in columns:
        dt = type(df[k].iloc[0])
        if 'float' in str(dt):
            df[k] = df[k].astype('float32')
        elif 'int' in str(dt):
            df[k] = df[k].astype('int32')
        elif dt == list:
            dt_ = type(df.iloc[0][k][0])
            if 'float' in str(dt_):
                df[k] = df[k].apply(lambda x : np.array(x, dtype=np.float32))
            elif 'int' in str(dt_):
                df[k] = df[k].apply(lambda x : np.array(x, dtype=np.int32))

In [6]:
def get_session_last_item(session_df):
    last_items = []
    num_sessions = len(session_df)
    for i in tqdm(range(num_sessions)):
        sess = session_df.iloc[i]
        sess_prev_items = sess['prev_items']
        
        product_list = sess_prev_items.strip('[]').split(' ')
        last_item = product_list[-1].strip("'\n")

        last_items.append(last_item)
    return last_items 

In [7]:
def get_co_graph_counts(session_last_items, merged_candidates_df, co_graph_dict):
    co_graph_count_list = []
    for idx, row in tqdm(merged_candidates_df.iterrows(), total=merged_candidates_df.shape[0]):
        sess_id = row['sess_id']
        product = row['product']
        last_item = session_last_items[sess_id]
        co_graph_count = co_graph_dict[last_item][product]
        co_graph_count_list.append(co_graph_count)
    return co_graph_count_list

In [8]:
def flatten_co_graph_dict(co_graph_dict):
    product_list = []
    neighbor_list = []
    counts_list = []
    for product in tqdm(co_graph_dict.keys(), total=len(co_graph_dict)):
        for neigh in co_graph_dict[product].keys():
            product_list.append(product)
            neighbor_list.append(neigh)
            counts_list.append(co_graph_dict[product][neigh])
    return pd.DataFrame({'product_' : product_list, 'neighbor' : neighbor_list, 'counts' : counts_list})

In [9]:
def normalize_co_graph_counts(merged_candidates_counts):
    # normalize co graph counts 
    # merged_candidates_counts_g = cudf.from_pandas(merged_candidates_counts)
    sessions_count_sum = merged_candidates_counts[['sess_id', 'counts']].groupby('sess_id').sum()
    sessions_count_sum.sort_index(inplace=True)

    sessions_count_sum = sessions_count_sum.to_pandas()

    candidates_count_sum = sessions_count_sum.loc[merged_candidates_counts['sess_id']].reset_index(drop=True)
    merged_candidates_counts['counts_cum'] = candidates_count_sum['counts']
    merged_candidates_counts['normalized_counts'] = merged_candidates_counts['counts'] / merged_candidates_counts['counts_cum']

    # del merged_candidates_counts_g
    # del sessions_count_sum_g
    

# Merge valid co-graph counts 

In [10]:
merged_candidates_feature_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/XGBoost/candidates_phase2/merged_candidates_150_feature.parquet'
train_sessions_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/data_for_recstudio/task1_data/task13_4_task1_train_sessions_phase2.csv'
valid_sessions_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/data_for_recstudio/task1_data/task13_4_task1_valid_sessions_phase2.csv'
test_sessions_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/raw_data/sessions_test_task1_phase2.csv'
product_data_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/raw_data/products_train.csv'

In [11]:
@lru_cache(maxsize=1)
def read_merged_candidates_feature():
    return pd.read_parquet(merged_candidates_feature_path)

@lru_cache(maxsize=1)
def read_product_data():
    return pd.read_csv(product_data_path)

@lru_cache(maxsize=1)
def read_train_data():
    return pd.read_csv(train_sessions_path)

@lru_cache(maxsize=1)
def read_valid_data():
    return pd.read_csv(valid_sessions_path)

@lru_cache(maxsize=1)
def read_test_data():
    return pd.read_csv(test_sessions_path)

In [12]:
merged_candidates_feature = read_merged_candidates_feature()

In [13]:
merged_candidates = merged_candidates_feature[['sess_id', 'sess_locale', 'product']]

In [14]:
train_sess_data = read_train_data()
valid_sess_data = read_valid_data()
test_sess_data = read_test_data()
product = read_product_data()

In [15]:
train_sess_item = get_sessions(train_sess_data, list_item=False)
valid_sess_item = get_sessions(valid_sess_data, test=True, list_item=False)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 261816/261816 [00:06<00:00, 39776.89it/s]


## bidirection

In [16]:
# bidirection
# valid is included in train 
co_occurence_dict_bi = get_co_occurence_dict(train_sess_item, bidirection=True, weighted=False)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3966659/3966659 [01:32<00:00, 42702.38it/s]


In [17]:
# only one arg, can't use another arg
def get_bi_valid_session_co_graph_candidates(sess_id):
    sess = valid_sess_item[sess_id]
    prev_items = set()
    cand_counter = Counter()
    for item in sess:
        if item in co_occurence_dict_bi and item not in prev_items:
            cand_counter = cand_counter + co_occurence_dict_bi[item]
            prev_items.add(item) # one time for every item
    for item in sess:
        if item in cand_counter:
            cand_counter.pop(item) # remove history items 

    return cand_counter

In [18]:
valid_co_graph_candidates = TFDataset.from_dict({'sess_id' : list(range(valid_sess_data.shape[0]))})

In [19]:
# about 1 mins
sess_ids = list(range(len(valid_sess_item)))
pool = multiprocessing.Pool(10)
bi_valid_sessions_counter = pool.map(get_bi_valid_session_co_graph_candidates, sess_ids)

In [20]:
valid_co_graph_candidates, len(bi_valid_sessions_counter)

(Dataset({
     features: ['sess_id'],
     num_rows: 261816
 }),
 261816)

In [21]:
all_items_co_graph_count_list = []
for row in tqdm(merged_candidates.itertuples(), total=merged_candidates.shape[0]):
    all_items_co_graph_count_list.append(bi_valid_sessions_counter[row.sess_id][row.product])
assert len(all_items_co_graph_count_list) == merged_candidates.shape[0]
merged_candidates['all_items_co_graph_count_0'] = all_items_co_graph_count_list

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 78842199/78842199 [02:14<00:00, 584770.14it/s]
/tmp/ipykernel_108423/4091948514.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates['all_items_co_graph_count_0'] = all_items_co_graph_count_list


In [22]:
count_sum_array = merged_candidates.groupby(by='sess_id')['all_items_co_graph_count_0'].sum().to_numpy()
assert len(count_sum_array[merged_candidates['sess_id']]) == merged_candidates.shape[0]
merged_candidates['normalized_all_items_co_graph_count_0'] = count_sum_array[merged_candidates['sess_id']]
merged_candidates['normalized_all_items_co_graph_count_0'] = merged_candidates['all_items_co_graph_count_0'] / merged_candidates['normalized_all_items_co_graph_count_0']

/tmp/ipykernel_108423/860000090.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates['normalized_all_items_co_graph_count_0'] = count_sum_array[merged_candidates['sess_id']]
/tmp/ipykernel_108423/860000090.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates['normalized_all_items_co_graph_count_0'] = merged_candidates['all_items_co_graph_count_0'] / merged_candidates['normalized_all_items_co_graph_count_0']


In [23]:
merged_candidates_feature['normalized_all_items_co_graph_count_0'] = merged_candidates['normalized_all_items_co_graph_count_0']
merged_candidates_feature['all_items_co_graph_count_0'] = merged_candidates['all_items_co_graph_count_0']

In [24]:
merged_candidates_feature.query('sess_id==20000').sort_values(by=['sasrec_scores_2'], ascending=False)[['product', 'sasrec_scores_2', 'all_items_co_graph_count_0', 'normalized_all_items_co_graph_count_0']][:25]

,product,sasrec_scores_2,all_items_co_graph_count_0,normalized_all_items_co_graph_count_0
5965883,B09ZKF77CC,13.667938,5,0.018116
5965840,B09G2YWKP3,13.521121,4,0.014493
5965875,B09Z2XHMS3,13.468659,4,0.014493
5965830,B09D415MK8,13.464596,2,0.007246
5965831,B09D443ZZ7,13.088786,3,0.010870
5965876,B09Z2XVBJQ,12.996532,3,0.010870
5965880,B09ZK5K3JQ,12.818775,6,0.021739
5965820,B09489RCTT,12.725793,4,0.014493
5965747,B07PBN67YJ,12.459669,1,0.003623
5965827,B099ZR7QL5,12.331591,3,0.010870


## uni and weighted

In [25]:
# weight 
# co_occurence_dict_bi_weight = get_co_occurence_dict(train_sess_item + valid_sess_item, bidirection=True, weighted=True)
co_occurence_dict_uni_weight = get_co_occurence_dict(train_sess_item, bidirection=False, weighted=True)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3966659/3966659 [01:16<00:00, 51943.54it/s]


In [26]:
merged_candidates = merged_candidates_feature[['sess_id', 'sess_locale', 'product']]

In [27]:
# only one arg, can't use another arg
def get_uni_wei_valid_session_co_graph_candidates(sess_id):
    sess = valid_sess_item[sess_id]
    prev_items = set()
    cand_counter = Counter()
    for item in sess:
        if item in co_occurence_dict_uni_weight and item not in prev_items:
            cand_counter = cand_counter + co_occurence_dict_uni_weight[item]
            prev_items.add(item) # one time for every item
    for item in sess:
        if item in cand_counter:
            cand_counter.pop(item) # remove history items 
    
    return cand_counter

In [28]:
valid_co_graph_candidates = TFDataset.from_dict({'sess_id' : list(range(valid_sess_data.shape[0]))})

In [29]:
# about 1 mins
sess_ids = list(range(len(valid_sess_item)))
pool = multiprocessing.Pool(10)
uni_wei_valid_sessions_counter = pool.map(get_uni_wei_valid_session_co_graph_candidates, sess_ids)

In [30]:
valid_co_graph_candidates, len(uni_wei_valid_sessions_counter)

(Dataset({
     features: ['sess_id'],
     num_rows: 261816
 }),
 261816)

In [31]:
all_items_co_graph_count_list = []
for row in tqdm(merged_candidates.itertuples(), total=merged_candidates.shape[0]):
    all_items_co_graph_count_list.append(uni_wei_valid_sessions_counter[row.sess_id][row.product])
assert len(all_items_co_graph_count_list) == merged_candidates.shape[0]
merged_candidates['all_items_co_graph_count_1'] = all_items_co_graph_count_list

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 78842199/78842199 [02:20<00:00, 562557.10it/s]
/tmp/ipykernel_108423/1702843849.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates['all_items_co_graph_count_1'] = all_items_co_graph_count_list


In [32]:
count_sum_array = merged_candidates.groupby(by='sess_id')['all_items_co_graph_count_1'].sum().to_numpy()
assert len(count_sum_array[merged_candidates['sess_id']]) == merged_candidates.shape[0]
merged_candidates['normalized_all_items_co_graph_count_1'] = count_sum_array[merged_candidates['sess_id']]
merged_candidates['normalized_all_items_co_graph_count_1'] = merged_candidates['all_items_co_graph_count_1'] / merged_candidates['normalized_all_items_co_graph_count_1']

/tmp/ipykernel_108423/1383280374.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates['normalized_all_items_co_graph_count_1'] = count_sum_array[merged_candidates['sess_id']]
/tmp/ipykernel_108423/1383280374.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates['normalized_all_items_co_graph_count_1'] = merged_candidates['all_items_co_graph_count_1'] / merged_candidates['normalized_all_items_co_graph_count_1']


In [33]:
merged_candidates_feature['normalized_all_items_co_graph_count_1'] = merged_candidates['normalized_all_items_co_graph_count_1']
merged_candidates_feature['all_items_co_graph_count_1'] = merged_candidates['all_items_co_graph_count_1']

In [34]:
merged_candidates_feature.query('sess_id==10000').sort_values(by=['sasrec_scores_2'], ascending=False)[['product', 'sasrec_scores_2', 'all_items_co_graph_count_1', 'normalized_all_items_co_graph_count_1']][:25]

,product,sasrec_scores_2,all_items_co_graph_count_1,normalized_all_items_co_graph_count_1
2980058,B0B3S4QXZ3,19.587433,5.400000,0.087206
2979948,B09SHX12P1,19.370029,6.571429,0.106124
2980034,B09ZVBXFJ2,18.312700,1.333333,0.021532
2979943,B09SHW3QP6,18.290028,2.350000,0.037951
2980035,B09ZVDCM2G,17.967262,2.895833,0.046765
2979911,B09NBZ82SM,17.482162,0.000000,0.000000
2979949,B09SHXN9T2,17.331429,0.090909,0.001468
2979941,B09SHVZ4DJ,17.320957,1.833333,0.029607
2979958,B09TPYW9C1,17.199915,0.783333,0.012650
2980031,B09ZJ2ZN7F,17.195761,1.500000,0.024224


## uni and dis=1

In [35]:
# max dis = 1 
# co_occurence_dict_bi_dis1 = get_co_occurence_dict(train_sess_item + valid_sess_item, bidirection=True, weighted=False, max_dis=1)
co_occurence_dict_uni_dis1 = get_co_occurence_dict(train_sess_item, bidirection=False, weighted=False, max_dis=1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3966659/3966659 [00:43<00:00, 91402.45it/s]


In [36]:
merged_candidates = merged_candidates_feature[['sess_id', 'sess_locale', 'product']]

In [37]:
# only one arg, can't use another arg
def get_uni_dis1_valid_session_co_graph_candidates(sess_id):
    sess = valid_sess_item[sess_id]
    prev_items = set()
    cand_counter = Counter()
    for item in sess:
        if item in co_occurence_dict_uni_dis1 and item not in prev_items:
            cand_counter = cand_counter + co_occurence_dict_uni_dis1[item]
            prev_items.add(item) # one time for every item
    for item in sess:
        if item in cand_counter:
            cand_counter.pop(item) # remove history items 
    
    return cand_counter

In [38]:
valid_co_graph_candidates = TFDataset.from_dict({'sess_id' : list(range(valid_sess_data.shape[0]))})

In [39]:
# about 1 mins
sess_ids = list(range(len(valid_sess_item)))
pool = multiprocessing.Pool(10)
uni_dis1_valid_sessions_counter = pool.map(get_uni_dis1_valid_session_co_graph_candidates, sess_ids)

In [40]:
valid_co_graph_candidates, len(uni_dis1_valid_sessions_counter)

(Dataset({
     features: ['sess_id'],
     num_rows: 261816
 }),
 261816)

In [41]:
all_items_co_graph_count_list = []
for row in tqdm(merged_candidates.itertuples(), total=merged_candidates.shape[0]):
    all_items_co_graph_count_list.append(uni_dis1_valid_sessions_counter[row.sess_id][row.product])
assert len(all_items_co_graph_count_list) == merged_candidates.shape[0]
merged_candidates['all_items_co_graph_count_2'] = all_items_co_graph_count_list

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 78842199/78842199 [02:22<00:00, 553277.11it/s]
/tmp/ipykernel_108423/3640618043.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates['all_items_co_graph_count_2'] = all_items_co_graph_count_list


In [42]:
count_sum_array = merged_candidates.groupby(by='sess_id')['all_items_co_graph_count_2'].sum().to_numpy()
assert len(count_sum_array[merged_candidates['sess_id']]) == merged_candidates.shape[0]
merged_candidates['normalized_all_items_co_graph_count_2'] = count_sum_array[merged_candidates['sess_id']]
merged_candidates['normalized_all_items_co_graph_count_2'] = merged_candidates['all_items_co_graph_count_2'] / merged_candidates['normalized_all_items_co_graph_count_2']

/tmp/ipykernel_108423/2314243688.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates['normalized_all_items_co_graph_count_2'] = count_sum_array[merged_candidates['sess_id']]
/tmp/ipykernel_108423/2314243688.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates['normalized_all_items_co_graph_count_2'] = merged_candidates['all_items_co_graph_count_2'] / merged_candidates['normalized_all_items_co_graph_count_2']


In [43]:
merged_candidates_feature['normalized_all_items_co_graph_count_2'] = merged_candidates['normalized_all_items_co_graph_count_2']
merged_candidates_feature['all_items_co_graph_count_2'] = merged_candidates['all_items_co_graph_count_2']

In [44]:
merged_candidates_feature.query('sess_id==150000').sort_values(by=['sasrec_scores_2'], ascending=False)[['product', 'sasrec_scores_2', 'all_items_co_graph_count_2', 'normalized_all_items_co_graph_count_2']][:25]

,product,sasrec_scores_2,all_items_co_graph_count_2,normalized_all_items_co_graph_count_2
45142626,B0B7B23YVQ,28.060143,39,0.520000
45142631,B0BBCMLDST,23.405308,8,0.106667
45142497,B01EDPMBSG,22.769426,7,0.093333
45142577,B09NTH72XH,20.928736,1,0.013333
45142620,B0B54ZNRP6,20.475601,4,0.053333
45142492,B013OKHAW8,18.864086,0,0.000000
45142551,B08D3LQW35,18.545700,0,0.000000
45142539,B07VNZVSLZ,18.489300,1,0.013333
45142526,B07PTH96MQ,18.476040,0,0.000000
45142489,B00NC4IGU2,18.374432,0,0.000000


In [45]:
cast_dtype(merged_candidates_feature, 
           ['all_items_co_graph_count_0', 'normalized_all_items_co_graph_count_0', 'all_items_co_graph_count_1', 'normalized_all_items_co_graph_count_1', 'all_items_co_graph_count_2', 'normalized_all_items_co_graph_count_2'])
merged_candidates_feature.to_parquet(merged_candidates_feature_path, engine='pyarrow')

In [46]:
merged_candidates_feature

,sess_id,sess_locale,product,target,sess_avg_price,product_price,sasrec_scores_3,normalized_sasrec_scores_3,sasrec_scores_2,normalized_sasrec_scores_2,...,co_graph_counts_1,normalized_co_graph_counts_1,co_graph_counts_2,normalized_co_graph_counts_2,cos_text_bert_scores,text_bert_scores,normalized_text_bert_scores,roberta_scores,normalized_roberta_scores,product_freq
0,0,DE,355165591X,0.0,43.256542,8.990000,2.230508,7.658405e-09,0.512931,1.377575e-09,...,0.000000,0.00000,0.0,0.000000,0.903757,378.286041,1.296655e-08,276.525787,7.975509e-07,48.0
1,0,DE,3833237058,0.0,43.256542,22.000000,9.605231,1.221631e-05,9.325538,9.255110e-06,...,0.090909,0.00083,0.0,0.000000,0.921604,387.624756,1.474268e-04,284.460052,2.226209e-03,80.0
2,0,DE,B00CIXSI6U,0.0,43.256542,6.470000,0.714114,1.681035e-09,-0.115904,7.345399e-10,...,0.000000,0.00000,0.0,0.000000,0.901061,374.802551,3.980740e-10,278.039612,3.624132e-06,7.0
3,0,DE,B00NVDOWUW,0.0,43.256542,11.990000,8.750996,5.199363e-06,8.507557,4.084482e-06,...,0.000000,0.00000,0.0,0.000000,0.927298,385.701782,2.154962e-05,285.239197,4.852260e-03,155.0
4,0,DE,B00NVDP3ZU,0.0,43.256542,22.990000,8.056712,2.596729e-06,5.898870,3.007453e-07,...,0.000000,0.00000,0.0,0.000000,0.930655,385.398499,1.591202e-05,284.763611,3.015780e-03,462.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78842194,261815,UK,B0BCX524Y6,0.0,9.383333,16.990000,6.813615,1.076201e-03,7.203015,4.597607e-04,...,0.000000,0.00000,0.0,0.000000,0.972680,438.956238,1.636532e-04,281.902344,1.476179e-04,7.0
78842195,261815,UK,B0BCX6QB4L,0.0,9.383333,10.990000,9.030836,9.881445e-03,10.123234,8.526421e-03,...,4.250000,0.01369,2.0,0.010638,0.972680,438.956238,1.636532e-04,281.902344,1.476179e-04,51.0
78842196,261815,UK,B0BFPJYXQL,0.0,9.383333,10.560000,0.796892,2.623396e-06,1.711608,1.895152e-06,...,0.000000,0.00000,0.0,0.000000,0.953467,430.164368,2.486932e-08,283.306732,6.012531e-04,7.0
78842197,261815,UK,B0BH3X67S3,0.0,9.383333,6.830000,4.250781,8.296004e-05,6.447586,2.159998e-04,...,0.000000,0.00000,0.0,0.000000,0.961829,434.029083,1.186011e-06,273.954742,5.218429e-08,37.0
